<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>LSTM and GRU on Sentiment Analysis</H1></u></center>

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, CuDNNLSTM, CuDNNGRU, Dropout
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
num_words = 20000

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [ ]:
print(len(X_train), 'train_data')
print(len(X_test), 'test_data')

In [ ]:
print(X_train[0])

In [ ]:
len(X_train[0])

## Hyperparameters:

In [ ]:
max_len = 256
embedding_size = 10
batch_size = 128
n_epochs = 10

## Creating Sequences

In [ ]:
pad  =  'pre' #'post'

In [ ]:
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

In [ ]:
X_train_pad[0]

## Creating the model:

In [ ]:
model = Sequential()

In [ ]:
#The input is a 2D tensor: (samples, sequence_length)
# this layer will return 3D tensor: (samples, sequence_length, embedding_dim)
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
#model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))

In [ ]:
model.add(CuDNNLSTM(128, return_sequences=False))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(1, activation='sigmoid', name='classification'))

In [ ]:
model.summary()

## Compiling the model:

In [ ]:
#optimizer = Adam(lr=0.001, decay=1e-6)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

## Callbacks:

In [ ]:
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

## Training the model:

In [ ]:
%%time
model.fit(X_train_pad, y_train,
          epochs=n_epochs,
          batch_size=batch_size, 
          validation_split=0.05,
          callbacks=[callback_early_stopping]
         )

## Testing the model:

In [ ]:
%%time
eval_ = model.evaluate(X_test_pad, y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

## Saving the model:

In [ ]:
model.save("..\data\models\{}".format('Sentiment-LSTM-GRU'))

## GRU model:

In [ ]:
model_GRU = Sequential()

In [ ]:
model_GRU.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [ ]:
model_GRU.add(CuDNNGRU(units=16, return_sequences=True))

In [ ]:
model_GRU.add(CuDNNGRU(units=8, return_sequences=True))

In [ ]:
model_GRU.add(CuDNNGRU(units=4, return_sequences=False))

In [ ]:
model_GRU.add(Dense(1, activation='sigmoid'))

In [ ]:
model_GRU.summary()

In [ ]:
model_GRU.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
%%time
model_GRU.fit(X_train_pad, y_train, validation_split=0.05, epochs=n_epochs, batch_size=batch_size)

In [ ]:
%%time
eval_GRU = model_GRU.evaluate(X_test_pad, y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_GRU[0]))
print("Accuracy: {0:.2%}".format(eval_GRU[1]))

## Examples of Mis-Classified Text

In [ ]:
#making predictions for the first 1000 test samples
y_pred = model.predict(X_test_pad[:1000])

In [ ]:
y_pred = y_pred.T[0]

In [ ]:
labels_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])

In [ ]:
true_labels = np.array(y_test[:1000])

In [ ]:
incorrect = np.where(labels_pred != true_labels)
incorrect = incorrect[0]

In [ ]:
print(incorrect)

In [ ]:
len(incorrect)

In [ ]:
idx = incorrect[1]
idx

In [ ]:
text = X_test[idx]
print(text)

In [ ]:
y_pred[idx]

In [ ]:
true_labels[idx]

## Converting integers in Text

In [ ]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

In [ ]:
word_index.items()

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index)

In [ ]:
def decode_index(text):
    return ' '.join([reverse_word_index.get(i) for i in text])

In [ ]:
decode_index(X_train[0])

In [ ]:
text_data = []
for i in range(len(X_train)):
    text_data.append(decode_index(X_train[i]))

In [ ]:
text_data[0]

## Embeddings

In [ ]:
layer_embedding = model.get_layer('layer_embedding')

In [ ]:
weights_embedding = layer_embedding.get_weights()[0]

In [ ]:
weights_embedding.shape

In [ ]:
weights_embedding[word_index.get('good')]

## Similar Words

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
def print_similar_words(word, metric='cosine'):
    
    token = word_index.get(word)

    embedding = weights_embedding[token]

    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    sorted_index = np.argsort(distances)
    
    sorted_distances = distances[sorted_index]
    
    sorted_words = [reverse_word_index[token] for token in sorted_index
                    if token != 0]

    def print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    N = 10

    print("Distance from '{0}':".format(word))

    print_words(sorted_words[0:N], sorted_distances[0:N])

    print("-------")

    print_words(sorted_words[-N:], sorted_distances[-N:])

In [ ]:
print_similar_words('good', metric='cosine')

## Reference:

https://keras.io/layers/recurrent/